In [1]:
import pandas

training_data = pandas.read_csv('data/train.csv')
print(training_data.head())

          ID      target  48df886f9  0deb4b6a8  34b15f335  a8cb14b00  \
0  000d6aaf2  38000000.0        0.0          0        0.0          0   
1  000fbd867    600000.0        0.0          0        0.0          0   
2  0027d6b71  10000000.0        0.0          0        0.0          0   
3  0028cbf45   2000000.0        0.0          0        0.0          0   
4  002a68644  14400000.0        0.0          0        0.0          0   

   2f0771a37  30347e683  d08d1fbe3  6ee66e115    ...      3ecc09859  \
0          0          0          0          0    ...            0.0   
1          0          0          0          0    ...            0.0   
2          0          0          0          0    ...            0.0   
3          0          0          0          0    ...            0.0   
4          0          0          0          0    ...            0.0   

   9281abeea  8675bec0b  3a13ed79a  f677d4d13  71b203550  137efaa80  \
0        0.0        0.0          0          0          0          0  

In [2]:
training_data.dtypes[0:5]

ID            object
target       float64
48df886f9    float64
0deb4b6a8      int64
34b15f335    float64
dtype: object

In [3]:
training_data.shape

(4459, 4993)

In [4]:
non_nan_column_counts = training_data.count(axis=0, numeric_only=True)
len(non_nan_column_counts[non_nan_column_counts != len(training_data)])

0

In [5]:
import numpy


training_data['log_target'] = numpy.log(training_data['target'] + 1.0)
assert ((numpy.exp(training_data['log_target']) - 1.0) - training_data['target'] < 0.0001).all()
training_data.dtypes.tail()

137efaa80       int64
fb36b89d9       int64
7e293fbaf       int64
9fc776466       int64
log_target    float64
dtype: object

In [6]:
training_data.shape

(4459, 4994)

In [7]:
training_data.drop(columns=['log_target', 'target', 'ID']).shape

(4459, 4991)

In [8]:
import sklearn.linear_model as linear_model
linear_regression = linear_model.LinearRegression()
linear_regression.fit(training_data.drop(columns=['log_target', 'target', 'ID']), training_data['log_target'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [9]:
training_data['log_prediction'] = linear_regression.predict(training_data.drop(columns=['log_target', 'target', 'ID']))
import sklearn.metrics as metrics
import math
math.sqrt(metrics.mean_squared_error(training_data['log_target'], training_data['log_prediction']))

0.08595782873918051

In [20]:
testing_data = pandas.read_csv('data/test.csv')

In [21]:
testing_data.iloc[0:2, 0:2]

,ID,48df886f9
0,000137c73,0.0
1,00021489f,0.0


In [22]:
non_nan_column_counts = testing_data.iloc[:, 1:].count()
assert len(non_nan_column_counts[non_nan_column_counts < len(testing_data)]) == 0

In [23]:
testing_data.shape

(49342, 4992)

In [24]:
log_targets = linear_regression.predict(testing_data.drop(columns=['ID']))
testing_data['log_target'] = log_targets
testing_data['log_target'] = testing_data['log_target'].clip(lower=0.0, upper=math.log(4600000000000.0 + 1.0))

In [25]:
testing_data['target'] = numpy.exp(testing_data['log_target']) - 1.0

In [28]:
testing_data[['ID', 'target']].to_csv('data/baseline_linear_regression.csv', index=False)